In [11]:
import torch
import torch.nn as nn
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T
from torch_geometric.nn import VGAE, GCNConv
from torch_geometric.utils import train_test_split_edges
import torch.nn.functional as F
from torch_geometric.nn.models import InnerProductDecoder, VGAE
from torch_geometric.nn.conv import GCNConv
from torch_geometric.utils import negative_sampling, remove_self_loops, add_self_loops
import os.path as osp
from torch.optim import Adam
from torch_geometric.transforms import RandomLinkSplit
from sklearn.cluster import KMeans
import torch_geometric

import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from utils.eval_metrics import *

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [12]:
dataset = 'Cora'
path = osp.join('..', 'data', dataset)
dataset = Planetoid(path, dataset, transform=T.NormalizeFeatures())
data = dataset[0]

In [13]:
class GCNEncoder(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GCNEncoder, self).__init__()
        self.gcn_shared = GCNConv(in_channels, hidden_channels)
        self.gcn_mu = GCNConv(hidden_channels, out_channels)
        self.gcn_logvar = GCNConv(hidden_channels, out_channels)

    def forward(self, x, edge_index):
        x = F.relu(self.gcn_shared(x, edge_index))
        mu = self.gcn_mu(x, edge_index)
        logvar = self.gcn_logvar(x, edge_index)
        return mu, logvar


class DeepVGAE(VGAE):
    def __init__(self, enc_in_channels, enc_hidden_channels, enc_out_channels):
        super(DeepVGAE, self).__init__(encoder=GCNEncoder(enc_in_channels,
                                                          enc_hidden_channels,
                                                          enc_out_channels),
                                       decoder=InnerProductDecoder())

    def forward(self, x, edge_index):
        z = self.encode(x, edge_index)
        adj_pred = self.decoder.forward_all(z)
        return adj_pred

    def loss(self, x, pos_edge_index, all_edge_index):
        z = self.encode(x, pos_edge_index)

        pos_loss = -torch.log(
            self.decoder(z, pos_edge_index, sigmoid=True) + 1e-15).mean()

        all_edge_index_tmp, _ = remove_self_loops(all_edge_index)
        all_edge_index_tmp, _ = add_self_loops(all_edge_index_tmp)

        neg_edge_index = negative_sampling(all_edge_index_tmp, z.size(0), pos_edge_index.size(1))
        neg_loss = -torch.log(1 - self.decoder(z, neg_edge_index, sigmoid=True) + 1e-15).mean()

        kl_loss = 1 / x.size(0) * self.kl_loss()

        return pos_loss + neg_loss + kl_loss

    def single_test(self, x, train_pos_edge_index, test_pos_edge_index, test_neg_edge_index):
        with torch.no_grad():
            z = self.encode(x, train_pos_edge_index)
        roc_auc_score, average_precision_score = self.test(z, test_pos_edge_index, test_neg_edge_index)
        return roc_auc_score, average_precision_score

In [14]:
model = DeepVGAE(enc_in_channels=data.num_features, enc_hidden_channels=128, enc_out_channels=64).to(device)
optimizer = Adam(model.parameters(), lr=0.01)

for epoch in range(1,751):
    model.train()
    optimizer.zero_grad()
    z = model.encode(data.x, data.edge_index)
    loss = model.recon_loss(z, data.edge_index) + (1 / data.num_nodes) * model.kl_loss()
    loss.backward()
    optimizer.step()

    if epoch % 50 == 0:
        model.eval()
        z = model.encode(data.x, data.edge_index)
        kmeans = KMeans(n_clusters=dataset.num_classes, n_init=10, random_state=0).fit(z.detach().cpu().numpy())
        predicted_labels = torch.tensor(kmeans.labels_, device=device)
        acc, nmi = eval_metrics(data.y, predicted_labels)
        print(f'Epoch: {epoch:03d}, Loss: {loss.item():.4f}, NMI: {nmi:.4f}, ACC: {acc:.4f}')

Epoch: 050, Loss: 1.3726, NMI: 0.1615, ACC: 0.3493
Epoch: 100, Loss: 1.0362, NMI: 0.3568, ACC: 0.4815
Epoch: 150, Loss: 0.9476, NMI: 0.4690, ACC: 0.6329
Epoch: 200, Loss: 0.9236, NMI: 0.4905, ACC: 0.6137
Epoch: 250, Loss: 0.9012, NMI: 0.5023, ACC: 0.6488
Epoch: 300, Loss: 0.8866, NMI: 0.5119, ACC: 0.6739
Epoch: 350, Loss: 0.8804, NMI: 0.5007, ACC: 0.6636
Epoch: 400, Loss: 0.8699, NMI: 0.4975, ACC: 0.6617
Epoch: 450, Loss: 0.8724, NMI: 0.4885, ACC: 0.6606
Epoch: 500, Loss: 0.8624, NMI: 0.4967, ACC: 0.6784
Epoch: 550, Loss: 0.8590, NMI: 0.4957, ACC: 0.6673
Epoch: 600, Loss: 0.8512, NMI: 0.4749, ACC: 0.6529
Epoch: 650, Loss: 0.8565, NMI: 0.4935, ACC: 0.6673
Epoch: 700, Loss: 0.8459, NMI: 0.4769, ACC: 0.6403
Epoch: 750, Loss: 0.8450, NMI: 0.4613, ACC: 0.5720


In [15]:
model.eval()
z = model.encode(data.x, data.edge_index).detach().cpu().numpy()

kmeans = KMeans(n_clusters=dataset.num_classes, n_init=10, random_state=0)
predicted_clusters = kmeans.fit_predict(z)

num_clusters = dataset.num_classes
cluster_adj_matrix = np.zeros((num_clusters, num_clusters))

for i in range(data.edge_index.size(1)):
    src, dest = data.edge_index[:, i]
    src_cluster = predicted_clusters[src.item()]
    dest_cluster = predicted_clusters[dest.item()]
    if src_cluster != dest_cluster:
        cluster_adj_matrix[src_cluster, dest_cluster] = 1
        cluster_adj_matrix[dest_cluster, src_cluster] = 1

cluster_edge_index = torch.tensor(np.array(np.nonzero(cluster_adj_matrix)), dtype=torch.long)
cluster_centers = torch.tensor(kmeans.cluster_centers_, dtype=torch.float)

clustered_data = torch_geometric.data.Data(x=cluster_centers, edge_index=cluster_edge_index)
print(clustered_data)

Data(x=[7, 64], edge_index=[2, 42])
